# MCQ (Multiple Choice Question) 평가 튜토리얼

## MCQDataset

이 튜토리얼에서는 Huggingface의 객관식 dataset을 불러와서 평가 후 재업로드하는 과정까지 경험해볼 것입니다.

### 1. 데이터셋 불러오기
먼저 HuggingFace Hub에서 데이터셋을 불러오는 방법을 알아보겠습니다:

In [1]:
from langmetrics.llmdataset import LLMDataset
from langmetrics.llmtestcase import LLMTestCase
from datasets import load_dataset
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
dataset = load_dataset('sickgpt/001_MedQA_raw')

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'expected_output', 'choices'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'expected_output', 'choices'],
        num_rows: 1273
    })
})

이제 LLMDataset을 이용해서 불러와봅시다.

먼저 LLMTestCase는 input, choices, expected_output을 고정으로 받습니다. 그런데 위에 Dataset은 input이 question이라는 열로 되어있네요. field_mapping 인자를 이용해서 column을 매핑해주겠습니다.

In [5]:
LLMTestCase.__annotations__

{'input': str,
 'output': typing.Optional[str],
 'expected_output': str,
 'context': typing.Optional[typing.List[str]],
 'retrieval_context': typing.Optional[typing.List[str]],
 'reasoning': typing.Optional[str],
 'choices': typing.Optional[str]}

In [6]:
# 예시 사용법
field_mapping = {
    'input': 'question',  # 데이터셋의 'question' 필드를 'input'으로 매핑
    'expected_output': 'expected_output',
    'choices': 'choices'
}

In [7]:
dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping)

In [8]:
print(len(dataset))

10178


In [9]:
test_dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping, split='test')

In [10]:
len(test_dataset)

1273

이제 evaluate을 진행해봅시다.

In [11]:
from langmetrics.llmfactory import LLMFactory
from langmetrics.config import ModelConfig

In [12]:
LLMFactory.get_model_list()

['gpt-4o',
 'gpt-4o-mini',
 'deepseek-v3',
 'deepseek-reasoner',
 'claude-3.7-sonnet',
 'claude-3.5-sonnet',
 'claude-3.5-haiku',
 'naver',
 'gemini-2.0-flash']

In [13]:
# 커스텀 모델 설정 생성
custom_config = ModelConfig(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    api_base="http://qwen3b:8000/v1",
    api_key='EMPTY',
    max_tokens=32000,
    seed=66,
    provider="openai"
)

In [14]:
# localllm은 서버를 local에서 실행시키기 때문에 부팅되는 시간이 존재합니다.
custom_llm = LLMFactory.create_llm(custom_config, temperature=1.0)

In [15]:
from langmetrics.llmfactory import LLMFactory
# LLM 모델 생성
gpt_4o_mini = LLMFactory.create_llm('gpt-4o-mini')

In [34]:
from langmetrics.metrics import MCQMetric
metric = MCQMetric(
    answer_model=custom_llm,
    template_language='en',  # 'ko' 또는 'en'
    generate_template_type='reasoning'  # 'reasoning' 또는 'only_answer'
)

async를 통해서 빠르게 추론을 할 것입니다.

In [35]:
import nest_asyncio
nest_asyncio.apply()

In [36]:
print(test_dataset[0])

LLMTestCase(input='A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?', output=None, expected_output='B', context=None, retrieval_context=None, reasoning=None, choices=['Disclose the error to the patient and put it in the operative report', 'Tell the attending that he cannot fail to disclose this mistake', 'Report the physician to the ethics committee', 'Refuse to dictate the operative report'])


In [69]:
results = await metric.ameasure(test_dataset[512:1000])

In [70]:
results

LLMDataset(Polars DataFrame with 488 rows)

In [71]:
# r1_results = await r1_metric.ameasure(test_dataset[:10])

약 1200개의 달하는 test를 단 30초만에 모두 추론한 것을 확인할 수 있습니다!

In [72]:
results.df['student_answer'][0]

'{\n    "reasoning": "The patient\'s thyroid-stimulating hormone (TSH) level of 2.0 μU/mL is lower than the normal range for TSH, which typically indicates hyperthyroidism. Since the patient is currently well-controlled with methimazole and asymptomatic, a lower TSH level suggests that methimazole is still effective in managing her hyperthyroidism. Discontinuing methimazole could potentially lead to the return of hyperthyroid symptoms. Adding glucocorticoids is not typically the first-line approach for managing hyperthyroidism, especially in a patient who is already on an antithyroid medication. Referring for radioiodine therapy is not recommended as an initial step since the patient is well-controlled and does not have any specific indications for this treatment. Therefore, the best course of action would be to continue methimazole.",\n    "answer": "A"\n}'

In [73]:
results.df

input,student_answer,teacher_answer,expected_output,context,retrieval_context,reasoning,choices,score,metadata
str,str,null,str,null,null,str,list[str],i64,struct[4]
"""A 35-year-old woman presents t…","""{ ""reasoning"": ""The patien…",null,"""B""",null,null,"""The patient's thyroid-stimulat…","[""Continue methimazole"", ""Discontinue methimazole, start propylthiouracil"", … ""Refer for radioiodine therapy""]",0,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{189,301,490},null}"
"""A 65-year-old man presents to …","""{ ""reasoning"": ""The patien…",null,"""D""",null,null,"""The patient's symptoms suggest…","[""Compression fracture"", ""Herniated nucleus pulposus"", … ""Spinal stenosis""]",1,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{150,367,517},null}"
"""A 3-year-old girl is brought t…","""{ ""reasoning"": ""<The patie…",null,"""A""",null,null,"""<The patient's symptoms sugges…","[""Anaphylactic transfusion reactions"", ""Cutaneous granulomas"", … ""Disseminated tuberculosis""]",0,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{156,367,523},null}"
"""An 8-year-old boy who recently…","""{ ""reasoning"": ""The patien…",null,"""D""",null,null,"""The patient presents with a pi…","[""Atypical lymphocytes on peripheral blood smear"", ""Starry sky appearance"", … ""Granulomas with giant cells""]",0,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{122,272,394},null}"
"""A 59-year-old man presents to …","""{ ""reasoning"": ""The patien…",null,"""D""",null,null,"""The patient's cough is not a k…","[""Change lisinopril to propanolol"", ""Change lisinopril to amlodipine"", … ""Change lisinopril to losartan""]",1,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{114,337,451},null}"
…,…,…,…,…,…,…,…,…,…
"""A 36-year-old woman, gravida 3…","""{ ""reasoning"": ""Given the …",null,"""C""",null,null,"""Given the mother's BMI of 33 k…","[""Polyhydramnios"", ""Acute respiratory distress syndrome"", … ""Placental abruption""]",0,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{173,344,517},null}"
"""The only immunoglobulin found …","""{ ""reasoning"": ""The only i…",null,"""C""",null,null,"""The only immunoglobulin that e…","[""Protect against invasive helminth infection"", ""Protect against viral infections"", … ""Provides the most specific recognition to circulating antigens in the bloodstream""]",0,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{76,179,255},null}"
"""A 47-year-old woman is brought…","""{ ""reasoning"": ""<pH 7.26 i…",null,"""D""",null,null,"""<pH 7.26 indicates metabolic a…","[""Diabetic ketoacidosis"", ""Diuretic overdose"", … ""Opioid overdose""]",1,"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{172,308,480},null}"


In [74]:
print(len(results))

488


In [75]:
print([i.score for i in results])

[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 

In [76]:
scores = sum([i.score for i in results]) / len(results)

In [77]:
print(scores)

0.4569672131147541


In [78]:
print(results[0])

LLMResult(input='A 35-year-old woman presents to her primary care provider concerned that she may be pregnant. She has a history of regular menstruation every 4 weeks that lasts about 4 days with mild to moderate bleeding, but she missed her last period 2 weeks ago. A home pregnancy test was positive. She has a 6-year history of hyperthyroidism that is well-controlled with daily methimazole. She is currently asymptomatic and has no complaints or concerns. A blood specimen is taken and confirms the diagnosis. Additionally, her thyroid-stimulating hormone (TSH) is 2.0 μU/mL. Which of the following is the next best step in the management of this patient?', student_answer='{\n    "reasoning": "The patient\'s thyroid-stimulating hormone (TSH) level of 2.0 μU/mL is lower than the normal range for TSH, which typically indicates hyperthyroidism. Since the patient is currently well-controlled with methimazole and asymptomatic, a lower TSH level suggests that methimazole is still effective in ma

In [79]:
results.df['metadata']

metadata
struct[4]
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{189,301,490},null}"
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{150,367,517},null}"
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{156,367,523},null}"
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{122,272,394},null}"
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{114,337,451},null}"
…
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{173,344,517},null}"
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{76,179,255},null}"
"{""en"",""Qwen/Qwen2.5-3B-Instruct"",{172,308,480},null}"
